# Information Extraction with GEPA

The project is from the DSPy Documentation [Link](https://dspy.ai/tutorials/gepa_facilitysupportanalyzer/)

This notebook demonstrates how to use the GEPA (Generative Prompt Adaptation) optimizer in DSPy to build and optimize an information extraction system. The goal is to analyze facility support messages and extract three key pieces of information:

1. **Urgency**: Classify messages as low, medium, or high urgency
2. **Sentiment**: Determine if the message is positive, neutral, or negative
3. **Categories**: Identify which facility management categories apply to each message

The notebook shows how to:
- Set up a DSPy module with multiple prediction tasks
- Evaluate the baseline performance
- Use GEPA to automatically optimize prompts through reflection and feedback
- Visualize the optimization process and results

The optimization process improves the model's accuracy from ~70% to ~86% by automatically refining the prompts based on feedback from the validation set.


In [1]:
%run supportvectors-common.ipynb


<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



## Configure Language Model

Initialize and configure the DSPy language model. Here we use GPT-4.1-nano with temperature=1 for more diverse outputs during optimization.


In [2]:
import dspy
lm = dspy.LM("openai/gpt-4.1-nano", temperature=1)
dspy.configure(lm=lm)

## Load and Prepare Dataset

Download the facility support analyzer dataset from the llama-prompt-ops repository and convert it into DSPy Examples. The dataset is split into training (33%), validation (33%), and test (34%) sets with a fixed random seed for reproducibility.


In [3]:
import requests
import dspy
import json
import random

def init_dataset():
    # Load from the url
    url = "https://raw.githubusercontent.com/meta-llama/llama-prompt-ops/refs/heads/main/use-cases/facility-support-analyzer/dataset.json"
    dataset = json.loads(requests.get(url).text)
    dspy_dataset = [
        dspy.Example({
            "message": d['fields']['input'],
            "answer": d['answer'],
        }).with_inputs("message")
        for d in dataset
    ]
    random.Random(0).shuffle(dspy_dataset)
    train_set = dspy_dataset[:int(len(dspy_dataset) * 0.33)]
    val_set = dspy_dataset[int(len(dspy_dataset) * 0.33):int(len(dspy_dataset) * 0.66)]
    test_set = dspy_dataset[int(len(dspy_dataset) * 0.66):]

    return train_set, val_set, test_set

## Initialize Dataset Splits

Load the dataset and display the size of each split (training, validation, and test sets).


In [4]:
train_set, val_set, test_set = init_dataset()

len(train_set), len(val_set), len(test_set)

(66, 66, 68)

## Examine Example Data

Display a sample input message and its corresponding gold standard answer to understand the data format and expected output structure.


In [5]:
print("Input Message:")
print(train_set[0]['message'])

print("\n\nGold Answer:")
for k, v in json.loads(train_set[0]['answer']).items():
    print(f"{k}: {v}")

Input Message:
Subject: Adjusting Bi-Weekly Cleaning Schedule for My Office

Dear ProCare Facility Solutions Support Team,

I hope this message finds you well. My name is Dr. Alex Turner, and I have been utilizing your services for my office space for the past year. I must say, your team's dedication to maintaining a pristine environment has been commendable and greatly appreciated.

I am reaching out to discuss the scheduling of our regular cleaning services. While I find the logistical challenges of coordinating these services intellectually stimulating, I believe we could optimize the current schedule to better suit the needs of my team and our workflow. Specifically, I would like to explore the possibility of adjusting our cleaning schedule to a bi-weekly arrangement, ideally on Tuesdays and Fridays, to ensure our workspace remains consistently clean without disrupting our research activities.

Previously, I have attempted to adjust the schedule through the online portal, but I enc

## Define DSPy Module

Create the main DSPy module that performs information extraction. This includes:
- Three signature classes for urgency, sentiment, and category classification
- A multi-module (MM) class that chains together three ChainOfThought predictors
- Each predictor analyzes the message and extracts its respective information


In [6]:
from typing import List, Literal


class FacilitySupportAnalyzerUrgency(dspy.Signature):
    """
    Read the provided message and determine the urgency.
    """
    message: str = dspy.InputField()
    urgency: Literal['low', 'medium', 'high'] = dspy.OutputField()

class FacilitySupportAnalyzerSentiment(dspy.Signature):
    """
    Read the provided message and determine the sentiment.
    """
    message: str = dspy.InputField()
    sentiment: Literal['positive', 'neutral', 'negative'] = dspy.OutputField()

class FacilitySupportAnalyzerCategories(dspy.Signature):
    """
    Read the provided message and determine the set of categories applicable to the message.
    """
    message: str = dspy.InputField()
    categories: List[Literal["emergency_repair_services", "routine_maintenance_requests", "quality_and_safety_concerns", "specialized_cleaning_services", "general_inquiries", "sustainability_and_environmental_practices", "training_and_support_requests", "cleaning_services_scheduling", "customer_feedback_and_complaints", "facility_management_issues"]] = dspy.OutputField()

class FacilitySupportAnalyzerMM(dspy.Module):
    def __init__(self):
        self.urgency_module = dspy.ChainOfThought(FacilitySupportAnalyzerUrgency)
        self.sentiment_module = dspy.ChainOfThought(FacilitySupportAnalyzerSentiment)
        self.categories_module = dspy.ChainOfThought(FacilitySupportAnalyzerCategories)
    
    def forward(self, message: str):
        urgency = self.urgency_module(message=message)
        sentiment = self.sentiment_module(message=message)
        categories = self.categories_module(message=message)

        return dspy.Prediction(
            urgency=urgency.urgency,
            sentiment=sentiment.sentiment,
            categories=categories.categories
        )

program = FacilitySupportAnalyzerMM()

## Define Evaluation Metrics

Implement scoring functions for each task (urgency, sentiment, categories) and a combined metric function. The metric computes accuracy for each component and returns the average as the overall score.


In [7]:
def score_urgency(gold_urgency, pred_urgency):
    """
    Compute score for the urgency module.
    """
    score = 1.0 if gold_urgency == pred_urgency else 0.0
    return score

def score_sentiment(gold_sentiment, pred_sentiment):
    """
    Compute score for the sentiment module.
    """
    score = 1.0 if gold_sentiment == pred_sentiment else 0.0
    return score

def score_categories(gold_categories, pred_categories):
    """
    Compute score for the categories module.
    Uses the same match/mismatch logic as category accuracy in the score.
    """
    correct = 0
    for k, v in gold_categories.items():
        if v and k in pred_categories:
            correct += 1
        elif not v and k not in pred_categories:
            correct += 1
    score = correct / len(gold_categories)
    return score

def metric(example, pred, trace=None, pred_name=None, pred_trace=None):
    """
    Computes a score based on agreement between prediction and gold standard for categories, sentiment, and urgency.
    Returns the score (float).
    """
    # Parse gold standard from example
    gold = json.loads(example['answer'])

    # Compute scores for all modules
    score_urgency_val = score_urgency(gold['urgency'], pred.urgency)
    score_sentiment_val = score_sentiment(gold['sentiment'], pred.sentiment)
    score_categories_val = score_categories(gold['categories'], pred.categories)

    # Overall score: average of the three accuracies
    total = (score_urgency_val + score_sentiment_val + score_categories_val) / 3

    return total

## Evaluate Baseline Performance

Run the initial evaluation on the test set to establish baseline performance before optimization. This gives us a starting point to measure improvement.


In [8]:
import dspy
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metric,
    num_threads=32,
    display_table=True,
    display_progress=True
)

evaluate(program)

Average Metric: 51.53 / 68 (75.8%): 100%|██████████| 68/68 [00:43<00:00,  1.57it/s]

2025/11/12 18:59:57 INFO dspy.evaluate.evaluate: Average Metric: 51.53333333333333 / 68 (75.8%)


,message,answer,urgency,sentiment,categories,metric
0,"Hey ProCare Support Team, Hope you all are doing great! My name is...","{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",medium,positive,"[sustainability_and_environmental_practices, general_inquiries]",✔️ [0.633]
1,"Hey ProCare Team, Hope you’re all doing well! My name’s Jake, and ...","{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,positive,[routine_maintenance_requests],✔️ [1.000]
2,"Subject: Assistance Needed for HVAC Maintenance Hi [Receiver], I h...","{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,neutral,[routine_maintenance_requests],✔️ [1.000]
3,Subject: A Green Inquiry from a Bill Maher Enthusiast Hey ProCare ...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,positive,[sustainability_and_environmental_practices],✔️ [1.000]
4,Subject: Inquiry on Sustainability Practices Dear ProCare Facility...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,neutral,"[sustainability_and_environmental_practices, general_inquiries]",✔️ [0.967]
...,...,...,...,...,...,...
63,Subject: Inquiry About Your Eco-Friendly Practices Dear ProCare Fa...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",medium,positive,"[sustainability_and_environmental_practices, general_inquiries]",✔️ [0.300]
64,Subject: Assistance Needed for Facility Management Issue Dear ProC...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",high,positive,[facility_management_issues],✔️ [0.667]
65,"Subject: Request for Training and Support Hi ProCare Support Team,...","{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",medium,positive,[training_and_support_requests],✔️ [0.667]
66,Subject: Concerns About Studio Maintenance and Rent Increase Dear ...,"{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",high,negative,"[routine_maintenance_requests, facility_management_issues, quality...",✔️ [0.300]


EvaluationResult(score=75.78, results=<list of 68 results>)

## Add Feedback Functions for GEPA

Extend the metric function to provide detailed feedback for each module. GEPA uses this feedback to guide prompt optimization. The feedback functions identify what was correct and incorrect, helping the optimizer understand how to improve each predictor.


In [9]:
import json
import dspy

def feedback_urgency(gold_urgency, pred_urgency):
    """
    Generate feedback for the urgency module.
    """
    score = 1.0 if gold_urgency == pred_urgency else 0.0
    if gold_urgency == pred_urgency:
        feedback = f"You correctly classified the urgency of the message as `{gold_urgency}`. This message is indeed of `{gold_urgency}` urgency."
    else:
        feedback = f"You incorrectly classified the urgency of the message as `{pred_urgency}`. The correct urgency is `{gold_urgency}`. Think about how you could have reasoned to get the correct urgency label."
    return feedback, score

def feedback_sentiment(gold_sentiment, pred_sentiment):
    """
    Generate feedback for the sentiment module.
    """
    score = 1.0 if gold_sentiment == pred_sentiment else 0.0
    if gold_sentiment == pred_sentiment:
        feedback = f"You correctly classified the sentiment of the message as `{gold_sentiment}`. This message is indeed `{gold_sentiment}`."
    else:
        feedback = f"You incorrectly classified the sentiment of the message as `{pred_sentiment}`. The correct sentiment is `{gold_sentiment}`. Think about how you could have reasoned to get the correct sentiment label."
    return feedback, score

def feedback_categories(gold_categories, pred_categories):
    """
    Generate feedback for the categories module.
    Uses the same match/mismatch logic as category accuracy in the score.
    """
    correctly_included = [k for k, v in gold_categories.items() if v and k in pred_categories]
    incorrectly_included = [k for k, v in gold_categories.items() if not v and k in pred_categories]
    incorrectly_excluded = [k for k, v in gold_categories.items() if v and k not in pred_categories]
    correctly_excluded = [k for k, v in gold_categories.items() if not v and k not in pred_categories]  # For completeness in accuracy check

    # Recompute category accuracy (matches score logic)
    score = (len(correctly_included) + len(correctly_excluded)) / len(gold_categories)

    if score == 1.0:
        fb_text = f"The category classification is perfect. You correctly identified that the message falls under the following categories: `{repr(correctly_included)}`."
    else:
        fb_text = f"The category classification is not perfect. You correctly identified that the message falls under the following categories: `{repr(correctly_included)}`.\n"
        if incorrectly_included:
            fb_text += f"However, you incorrectly identified that the message falls under the following categories: `{repr(incorrectly_included)}`. The message DOES NOT fall under these categories.\n"
        if incorrectly_excluded:
            prefix = "Additionally, " if incorrectly_included else "However, "
            fb_text += f"{prefix}you didn't identify the following categories that the message actually falls under: `{repr(incorrectly_excluded)}`.\n"
        fb_text += "Think about how you could have reasoned to get the correct category labels."
    return fb_text, score

def metric_with_feedback(example, pred, trace=None, pred_name=None, pred_trace=None):
    """
    Computes a score based on agreement between prediction and gold standard for categories, sentiment, and urgency.
    Optionally provides feedback text for a specific predictor module, using the same comparison logic as the score.
    Returns a dspy.Prediction with score (float) and feedback (str).
    """
    # Parse gold standard from example
    gold = json.loads(example['answer'])

    # Compute feedback and scores for all modules
    fb_urgency, score_urgency = feedback_urgency(gold['urgency'], pred.urgency)
    fb_sentiment, score_sentiment = feedback_sentiment(gold['sentiment'], pred.sentiment)
    fb_categories, score_categories = feedback_categories(gold['categories'], pred.categories)

    # Overall score: average of the three accuracies
    total = (score_urgency + score_sentiment + score_categories) / 3

    if pred_name is None:
        return total

    elif pred_name == 'urgency_module.predict':
        feedback = fb_urgency
    elif pred_name == 'sentiment_module.predict':
        feedback = fb_sentiment
    elif pred_name == 'categories_module.predict':
        feedback = fb_categories

    return dspy.Prediction(score=total, feedback=feedback)

## Initialize GEPA Optimizer

Set up the GEPA optimizer with the feedback-enabled metric. GEPA will use reflection to iteratively improve prompts. The optimizer is configured with auto="heavy" for thorough optimization, multiple threads for parallel evaluation, and a reflection language model for generating improved prompts.


In [10]:
from dspy import GEPA

optimizer = GEPA(
    metric=metric_with_feedback,
    auto="heavy",
    num_threads=32,
    track_stats=True,
    use_merge=False,
    reflection_lm=dspy.LM(model="gpt-5", temperature=1.0, max_tokens=32000)
)

## Run GEPA Optimization

Compile and optimize the program using GEPA. This process will:
- Generate candidate prompt variations
- Evaluate them on the training and validation sets
- Use feedback to refine prompts through reflection
- Track the best performing versions

This may take several minutes as it explores many prompt variations.


In [11]:
optimized_program = optimizer.compile(
    program,
    trainset=train_set,
    valset=val_set,
)

2025/11/12 18:59:57 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 2632 metric calls of the program. This amounts to 19.94 full evals on the train+val set.
2025/11/12 18:59:57 INFO dspy.teleprompt.gepa.gepa: Using 66 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.
GEPA Optimization:   0%|          | 0/2632 [00:00<?, ?rollouts/s]2025/11/12 19:00:32 INFO dspy.evaluate.evaluate: Average Metric: 46.3 / 66 (70.2%)
2025/11/12 19:00:32 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.7015151515151514
GEPA Optimization:   3%|▎         | 66/2632 [00:35<22:42,  1.88rollouts/s]2025/11/12 19:00:32 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.70151515151515

Average Metric: 2.33 / 3 (77.8%): 100%|██████████| 3/3 [00:11<00:00,  3.95s/it]

2025/11/12 19:00:44 INFO dspy.evaluate.evaluate: Average Metric: 2.333333333333333 / 3 (77.8%)


2025/11/12 19:01:24 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for urgency_module.predict: Task: Determine the urgency of a single provided message.

Context:
- Messages are from clients of ProCare Facility Solutions (facility management, cleaning, HVAC, safety/quality).
- Typical topics: cleaning/maintenance quality, HVAC performance/maintenance, safety concerns, scheduling, or general inquiries (e.g., eco-friendly practices).

Input format:
- One field named "message" containing the full email (subject and body).

Output format:
- Provide exactly two top-level fields:
  - reasoning: 1–3 concise sentences explaining why you chose the urgency, based strictly on message content.
  - urgency: one of [low, medium, high, emergency]

How to classify urgency:
- emergency:
  - There is an immediate danger to life, health, or property, or an active critical failure requiring immediate action now.
  - Examples: active water leak/flood, fire/smoke/burning smell, gas/CO suspic

Average Metric: 2.60 / 3 (86.7%): 100%|██████████| 3/3 [00:10<00:00,  3.57s/it]

...

2025/11/12 20:58:35 INFO dspy.evaluate.evaluate: Average Metric: 2.6 / 3 (86.7%)


2025/11/12 20:59:31 INFO dspy.teleprompt.gepa.gepa: Iteration 72: Proposed new text for categories_module.predict: You are a multi-label message classifier. Your job is to read a single incoming message and return all applicable labels from the fixed taxonomy below, plus a brief justification.

Context and domain
- The sender is contacting ProCare Facility Solutions, a facility management company that offers:
  - Cleaning services (including deep/specialized/tailored cleanings).
  - HVAC/facilities maintenance (including routine/preventative maintenance).
- Many inquiries focus on:
  - Eco-friendly/“green” cleaning products and practices (e.g., Green Seal, EPA Safer Choice).
  - Specialized/tailored cleaning needs (e.g., artist studios).
  - Routine (preventative) HVAC maintenance and non-urgent performance issues.

Taxonomy (use only these exact labels)
- general_inquiries: The sender is primarily asking for information, details, or clarification about services/products, without placi

Average Metric: 2.57 / 3 (85.6%): 100%|██████████| 3/3 [00:05<00:00,  1.76s/it]

2025/11/12 20:59:51 INFO dspy.evaluate.evaluate: Average Metric: 2.5666666666666664 / 3 (85.6%)


2025/11/12 21:00:23 INFO dspy.teleprompt.gepa.gepa: Iteration 73: Proposed new text for urgency_module.predict: Task
- Determine the urgency of a single provided message.

Context
- Messages are from clients of ProCare Facility Solutions (facility management, cleaning, HVAC, safety/quality).
- Typical topics: cleaning/maintenance quality, HVAC performance/maintenance, safety concerns, scheduling, general inquiries (including eco-friendly practices).
- Apply the same criteria to residential and commercial clients; domain remains facilities services.

Input
- One field named "message" containing the full email (subject and body).

Output
- Provide exactly two top-level fields:
  - reasoning: 1–3 concise sentences explaining why you chose the urgency, based strictly on the message content.
  - urgency: one of [low, medium, high, emergency]
- Do not include any extra fields, formatting, recommendations, or actions.

How to classify urgency

1) emergency
- Definition: Immediate danger to li

## Inspect Optimized Prompts

Display the optimized prompts that GEPA generated for each predictor. These prompts have been refined through the optimization process and should be more effective than the original signatures.


In [12]:
for name, pred in optimized_program.named_predictors():
    print("================================")
    print(f"Predictor: {name}")
    print("================================")
    print("Prompt:")
    print(pred.signature.instructions)
    print("*********************************")

Predictor: urgency_module.predict
Prompt:
Task
Classify the urgency of a single client email.

Domain context
- Sender is a client of ProCare Facility Solutions (facility management, cleaning, HVAC, safety/quality).
- Common topics: cleaning/maintenance quality, HVAC performance/maintenance, safety concerns, security/space coordination, scheduling/events, and general inquiries (e.g., sustainability/eco-friendly practices).

Input
- One field named "message" containing the full email (subject and body). Subject lines may contain useful urgency signals.

Output
- Provide exactly two top-level fields:
  - reasoning: 1–3 concise sentences explaining why you chose the urgency, based strictly on the message content (no advice, no next steps).
  - urgency: exactly one of [low, medium, high, emergency].
- Do not include any other fields or formatting.

How to decide urgency
Base the decision on explicit safety risk, system failure, and stated timelines. If details are insufficient to infer ris

## Evaluate Optimized Program

Run final evaluation on the test set using the optimized program to confirm the performance improvement.


In [13]:
evaluate(optimized_program)

Average Metric: 58.10 / 68 (85.4%): 100%|██████████| 68/68 [00:54<00:00,  1.25it/s]

2025/11/12 21:01:32 INFO dspy.evaluate.evaluate: Average Metric: 58.1 / 68 (85.4%)


,message,answer,urgency,sentiment,categories,metric
0,"Hey ProCare Support Team, Hope you all are doing great! My name is...","{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,positive,"[sustainability_and_environmental_practices, general_inquiries]",✔️ [0.967]
1,"Hey ProCare Team, Hope you’re all doing well! My name’s Jake, and ...","{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,positive,[routine_maintenance_requests],✔️ [1.000]
2,"Subject: Assistance Needed for HVAC Maintenance Hi [Receiver], I h...","{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,neutral,[routine_maintenance_requests],✔️ [1.000]
3,Subject: A Green Inquiry from a Bill Maher Enthusiast Hey ProCare ...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,positive,"[sustainability_and_environmental_practices, general_inquiries]",✔️ [0.967]
4,Subject: Inquiry on Sustainability Practices Dear ProCare Facility...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,neutral,"[sustainability_and_environmental_practices, general_inquiries]",✔️ [0.967]
...,...,...,...,...,...,...
63,Subject: Inquiry About Your Eco-Friendly Practices Dear ProCare Fa...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,neutral,"[sustainability_and_environmental_practices, general_inquiries]",✔️ [0.967]
64,Subject: Assistance Needed for Facility Management Issue Dear ProC...,"{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",medium,positive,[facility_management_issues],✔️ [1.000]
65,"Subject: Request for Training and Support Hi ProCare Support Team,...","{""categories"": {""routine_maintenance_requests"": false, ""customer_f...",low,positive,"[training_and_support_requests, general_inquiries]",✔️ [0.967]
66,Subject: Concerns About Studio Maintenance and Rent Increase Dear ...,"{""categories"": {""routine_maintenance_requests"": true, ""customer_fe...",medium,neutral,[routine_maintenance_requests],✔️ [0.900]


EvaluationResult(score=85.44, results=<list of 68 results>)

## Performance Improvement Summary

Display the improvement in accuracy from baseline (75.78%) to optimized (85.44%) performance.

### Comparison: GEPA vs MIPROv2

| Optimizer | Baseline | Optimized | Improvement |
|-----------|----------|-----------|-------------|
| GEPA | 75.78% | 85.44% | +9.66 pp |
| MIPROv2 | 75.78% | 82.55% | +6.77 pp |

*pp = percentage points*